*https://www.kaggle.com/eddiery/*
## House Prices - Advanced Regression Techniques | Kaggle Comptetion

### Comptetion Description and challenge
Ask a home buyer to describe their dream house, and they probably won't begin with the height of the basement ceiling or the proximity to an east-west railroad. But this playground competition's dataset proves that much more influences price negotiations than the number of bedrooms or a white-picket fence.

With 79 explanatory variables describing (almost) every aspect of residential homes in Ames, Iowa, this competition challenges you to predict the final price of each home.

To solve this challange I'm going to use Linear Regression Models using diffrent type of approaches with Cross Validtation and different Feature Selection Models

### Imports and Definitions


First we have to update some libraries

In [1]:
!pip install -U scikit-learn
!pip install --upgrade plotly
!pip uninstall matplotlib -Y
!pip install matplotlib

### Imports

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
from scipy import stats
from scipy.stats import norm

import plotly.express as px
import plotly.graph_objects as go


from pandas_profiling import ProfileReport

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# sklearn imports
import sklearn
from sklearn import metrics
from sklearn import datasets
from sklearn import pipeline


from sklearn import preprocessing
from sklearn import model_selection

from sklearn import metrics
from sklearn import pipeline
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import neural_network
from sklearn import model_selection
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import RFE
from sklearn.model_selection import LeavePOut
from sklearn.feature_selection import RFECV
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import KFold
from tqdm.auto import tqdm


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
# Turn on autocomplete
%config Completer.use_jedi = False

In [4]:
data_path = '../input/house-prices-advanced-regression-techniques/train.csv'
test_path = '../input/house-prices-advanced-regression-techniques/test.csv'
submission_path = '../input/house-prices-advanced-regression-techniques/sample_submission.csv'

df_original = pd.read_csv(data_path)
test_df = pd.read_csv(test_path)
submission_df = pd.read_csv(submission_path)
display(df_original)

## Data Processing

Let's have a look on our data, we have 81 different features, we should examine our data and check for missing valuses, and understand which features are the best before we start train models.

In [5]:
df_original.info()

Check for empty values

In [6]:
print("Check if there are columns with empty values")
display(df_original.isna().any())

print(f'There are {len(np.where(df_original.isnull())[0])} empty values in the dataframe')

# count empty values in each column
def count_empty_values_in_each_column(df: pd.DataFrame):
  print('empty values')
  print('------------\n')
  
  for col in df.columns:
    if df[col].isna().sum() > 0:
        print(f"{col}: {df[col].isna().sum()}")

count_empty_values_in_each_column(df_original)

We can see that we have some features with missing values. 
For example MiscFeature has 1406 miss values out of 1460 samples, this is a lot and probably should be dropped.  
But before deciding that we should check out corrletion.

#### Correlation Map

In [7]:
plt.figure(figsize=(12,10))
cor = np.abs(df_original.corr())
sns.heatmap(cor, cmap=plt.cm.Reds, vmin=0, vmax=1, square=True)
plt.show()

We can see that most of those features has low correlation with our targer variable - 'SalePrice'.  
It probably should be best to drop those features and focus on the highest corrlated features.

In [8]:
df_cp = df_original.copy()
df_cp

## Feature Engeeniring

### First let's deal with our missing data

In [9]:
#missing data
total = df_cp.isnull().sum().sort_values(ascending=False)
percent = (df_cp.isnull().sum()/df_cp.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

We got a table of all the features with missing data in them.  
As I said before this features don't have strong corrleation, and we have a lot of other good featrues with strong correltion so it would be the best to drop them.  
About the one row with 'Elecrtical' missing in it, we can drop the row instead, one row in a quite big dataset won't make a big difference.

In [10]:
# dealing with missing data
print((missing_data[missing_data['Total'] > 0]).index)
df_cp = df_cp.drop((missing_data[missing_data['Total'] > 1]).index,1)
df_cp = df_cp.drop(df_cp.loc[df_cp['Electrical'].isnull()].index)
df_cp = df_cp.drop(['Id'], axis=1)
df_cp.reset_index(drop=True, inplace=True)
df_cp

In [11]:
#check that we indeed have no missing values left
count_empty_values_in_each_column(df_cp)

Target variable distribution

In [12]:
#histogram and normal probability plot
sns.distplot(df_cp['SalePrice'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_cp['SalePrice'], plot=plt)

In [13]:
# df_cp['SalePrice'] = np.log(df_cp['SalePrice'])

Before we start creating new features let's have a look on the top most correlated features.

In [14]:
#saleprice correlation matrix
plt.figure(figsize=(12,10))
k = 10 #number of variables for heatmap
cols = cor.nlargest(k, 'SalePrice')['SalePrice'].index
cm = np.corrcoef(df_cp[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, fmt='.2f', yticklabels=cols.values, xticklabels=cols.values)
plt.show()

We can see now that TotalBsmtSF and 1stFlrSF have strong corrleation.
TotalBsmtSF: Total square feet of basement area.  
1stFlrSF: First Floor square feet.  
These features are connected and we know we have another feature that simmilart to them which is 2ndFlrSF.  
2ndFlrSF: Second floor square feet.  
These 3 features can be combined and give us a new feature - Total area of the house.  
Also we have YearBuild, which is also have simmilar feature - YrSold. 
With both of these feature we can get the age of the house when it was sold: YrSold - YearBuild



#### Creating New Features

In [15]:
df_train = df_cp.copy()
df_train['TotalSF'] = df_train['TotalBsmtSF'] + df_train['1stFlrSF'] + df_train['2ndFlrSF']
df_train['HouseAge'] = df_train['YrSold'] - df_train['YearBuilt']

In [16]:
sns.scatterplot('TotalSF', 'SalePrice', data=df_train, palette='Set2', color="teal")
plt.show()

In [17]:
sns.scatterplot('HouseAge', 'SalePrice', data=df_train, palette='Set2', color="teal")
plt.show()

This new features indeed has a good corolletion with the targert variable  
We can learn that as long as the house age is lower the price is higher  
Also as long as the total surface is higher the price is higher.  
These two conclusions make sense.

### Cross-Validation
For spliting our data into train and valiation we can  
For spliting our data into train and valiation we can  use CV (Cross-Validation) instead of breaking the data to train-validation splits.  
When we use CV, we have better predictions of the test results.  
It is similar to splitting the data, but we make sure that our split won't affect our result (we try a few possible splits).  
We can use 2 CV methods:  
      1. KFold  
      2.LPO (Leave P out)  
We will use KFold when we want speed.  
We will use LPO when we want to be more precise and better predict the test score.  

In [19]:
# divide the data to features and target
t = df_train['SalePrice'].copy()
X = df_train.drop(['SalePrice'], axis=1)
print('t')
display(t)
print()
print('X')
display(X)

In [20]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=1)


In [21]:
def find_generator_len(generator, use_pbar=True):
    i = 0
    
    if use_pbar:
        pbar = tqdm(desc='Calculating Length', ncols=1000, bar_format='{desc}{bar:10}{r_bar}')

    for a in generator:
        i += 1

        if use_pbar:
            pbar.update()

    if use_pbar:
        pbar.close()

    return i

Let's create a method that gets: data and model and returns R2 score and MSE loss

In [22]:
# calculate score and loss from cv (KFold or LPO) and display graphs
def get_cv_score_and_loss(X, t, model, k=None, p=None, show_score_loss_graphs=False, use_pbar=True):
    scores_losses_df = pd.DataFrame(columns=['fold_id', 'split', 'score', 'loss'])

    if k is not None:
        cv = KFold(n_splits=k, shuffle=True, random_state=1)
    elif p is not None:
        cv = LeavePOut(p)
    else:
        raise ValueError('you need to specify k or p in order for the cv to work')

    if use_pbar:
        pbar = tqdm(desc='Computing Models', total=find_generator_len(cv.split(X)))

    for i, (train_ids, val_ids) in enumerate(cv.split(X)):
        X_train = X.loc[train_ids]
        t_train = t.loc[train_ids]
        X_val = X.loc[val_ids]
        t_val = t.loc[val_ids]

        model.fit(X_train, t_train)

        y_train = model.predict(X_train)
        y_val = model.predict(X_val)
        scores_losses_df.loc[len(scores_losses_df)] = [i, 'train', model.score(X_train, t_train), mean_squared_error(t_train, y_train, squared=False)]
        scores_losses_df.loc[len(scores_losses_df)] = [i, 'val', model.score(X_val, t_val), mean_squared_error(t_val, y_val, squared=False)]

        if use_pbar:
            pbar.update()

    if use_pbar:
        pbar.close()

    val_scores_losses_df = scores_losses_df[scores_losses_df['split']=='val']
    train_scores_losses_df = scores_losses_df[scores_losses_df['split']=='train']

    mean_val_score = val_scores_losses_df['score'].mean()
    mean_val_loss = val_scores_losses_df['loss'].mean()
    mean_train_score = train_scores_losses_df['score'].mean()
    mean_train_loss = train_scores_losses_df['loss'].mean()

    if show_score_loss_graphs:
        fig = px.line(scores_losses_df, x='fold_id', y='score', color='split', title=f'Mean Val Score: {mean_val_score:.2f}, Mean Train Score: {mean_train_score:.2f}')
        fig.show()
        fig = px.line(scores_losses_df, x='fold_id', y='loss', color='split', title=f'Mean Val Loss: {mean_val_loss:.2f}, Mean Train Loss: {mean_train_loss:.2f}')
        fig.show()

    return mean_val_score, mean_val_loss, mean_train_score, mean_train_loss

In [23]:
# determine categorical and numerical features
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object', 'bool']).columns
all_cols = np.array(X.columns)

print('Numerical Cols:', numerical_cols)
print('Categorical Cols:', categorical_cols)
print('All Cols:', all_cols)

In [24]:
# use column transformer to insert different transformers for each column
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer([
    ("encoding", OneHotEncoder(sparse=False, handle_unknown='ignore'), categorical_cols),
    ("standard", StandardScaler(), numerical_cols)])
model_pipe = make_pipeline(ct, SGDRegressor(random_state=1))
val_score, val_loss, train_score, train_loss = get_cv_score_and_loss(X, t, model_pipe, k=10, show_score_loss_graphs=True)
print(f'mean cv val score: {val_score:.2f}\nmean cv val loss {val_loss:.2f}')
print(f'mean cv val score: {train_score:.2f}\nmean cv val loss {train_loss:.2f}')

#### Feature Selection

We want to choose the best features for our use case.  
We have learned 3 methods of Feature Selection:  
1. Forward Feature Selection  
2. Backward Feature Selection  
3. Hybrid Feature Selection     
In Forward Feature Selection we start from zero features and add features until we reach the number of maximum features or until we reach the best score.  
In Backward Feature Selection we start from the full feature set and remove features until we reach the number of minimum features or until we reach the best score.  
In Hybrid Feature Selection, we start from zero features and add/remove features until we reach the best score.  
  
We will use Scikit-learn RFE that is based on the Backward Feature Selection.  
We will choose the best features of this dataset with this method.  

In [25]:
#  choose the best features of this dataset with SGDRegressor

ct_encoded = ColumnTransformer([
                ("encoding", OrdinalEncoder(), categorical_cols),
                ("standard", StandardScaler(), numerical_cols)])
X_encoded = pd.DataFrame(ct_encoded.fit_transform(X, t), columns=all_cols)

selector = RFECV(SGDRegressor(random_state=1), cv=RepeatedKFold(n_splits=5, n_repeats=10, random_state=1)).fit(X_encoded, t)
top_features_df = X_encoded.loc[:, selector.support_]
num_features = len(selector.support_)
print('_______', num_features, ' most segnificant features are:_______')
display(top_features_df.columns)


In [26]:
# best_features = ['MSZoning', 'Street', 'LandSlope', 'Condition1', 'Condition2',
#        'RoofStyle', 'RoofMatl', 'ExterQual', 'ExterCond', 'Foundation',
#        'Heating', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional',
#        'PavedDrive', 'SaleType', 'SaleCondition', 'LotArea', 'OverallCond',
#        'YearBuilt', 'YearRemodAdd', 'BsmtFinSF2', '1stFlrSF', '2ndFlrSF',
#        'LowQualFinSF', 'GrLivArea', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
#        'GarageArea', 'HouseAge']
best_features = top_features_df.columns.tolist()
print(best_features)

#### Testing the best features we got

In [27]:
X_test_temp = df_train.copy()
X_test_temp.drop('SalePrice',axis=1)

X_test_temp = df_train[best_features]

t_train_temp = df_train['SalePrice']

    
best_numerical_cols = X_test_temp.select_dtypes(include=['int64', 'float64']).columns
best_categorical_cols = X_test_temp.select_dtypes(include=['object', 'bool']).columns
best_all_cols = np.array(X_test_temp.columns)


ct = ColumnTransformer([
    ("encoding", OneHotEncoder(sparse=False, handle_unknown='ignore'), best_categorical_cols),
    ("standard", StandardScaler(), best_numerical_cols)])

model_pipe = make_pipeline(ct, SGDRegressor(random_state=1))

val_score, val_loss, train_score, train_loss = get_cv_score_and_loss(X_test_temp, t_train_temp, model_pipe, k=10, show_score_loss_graphs=True)
X_test_temp

#### Prediction on train  
After we trained and tested the validation data, we need to test on the original training data before we test the submission.

In [28]:
SGD_reg_original = pipeline.make_pipeline(ct, linear_model.SGDRegressor(random_state=1)).fit(X_test_temp, t_train_temp)

y_train = SGD_reg_original.predict(X_test_temp)

In [29]:
print('Accuracy score on train', SGD_reg_original.score(X_test_temp, t_train_temp))
print('RMSE score on train',(mean_squared_error(t_train_temp, y_train, squared=False)))


We are getting nice results in with this model.  
We can try and check if we can improve it with Regultiazations and Hyper Paramter Serach.

#### Regularizations
When we have a case of High-Variance, we can use Regularizations, to reduce it.  
With regularization, we can control the size of the model weights and thus control the variance.  
We control the size of the model weights by adding a term to the loss function.  
This term is called a penalty to high weights.  
We have learned about three regularization techniques:  
L1 (also called Lasso).  
L2 (also called Ridge).  
Elastic Net (a combination of Lasso and Ridge).  

In [30]:
# print lasso, ridge and elasticnet scores as regression 
from sklearn.model_selection import cross_val_score

sgd_lasso_reg = SGDRegressor(penalty='l1', random_state=1)
sgd_ridge_reg = SGDRegressor(penalty='l2', random_state=1)
sgd_elastic_reg = SGDRegressor(penalty='elasticnet', random_state=1)
# X_encoded = standartize_and_dummie_encoding(X_best)

print("R2 score for regression:")
print('sgd_lasso', cross_val_score(make_pipeline(ct, sgd_lasso_reg), X_test_temp, t_train_temp, cv=10).mean())
print('sgd_ridge', cross_val_score(make_pipeline(ct, sgd_ridge_reg), X_test_temp, t_train_temp, cv=10).mean())
print('sgd_elastic', cross_val_score(make_pipeline(ct, sgd_elastic_reg), X_test_temp, t_train_temp, cv=10).mean())

These methods gives us less accurate results than before

In [31]:
numerical_cols = X_test_temp.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X_test_temp.select_dtypes(include=['object', 'bool']).columns
all_cols = np.array(X_test_temp.columns)


ct = ColumnTransformer([
    ("encoding", OneHotEncoder(sparse=False, handle_unknown='ignore'), categorical_cols),
    ("standard", StandardScaler(), numerical_cols)])

In [32]:
X_test_temp_cp = X_test_temp.copy()
X_test_temp_cp.reset_index(drop=True, inplace=True)
X_test_temp_cp


#### Hyper Paramter Search
Most of our models have a lot of parameters that can be adjusted.  
Each parameter value can make our model better (or worse).  
We want to be able to find the best hyperparameters for our models.  
We have two approaches:  
Grid Search   
Random Search  
  
Grid Search  
When we want to check every parameter possible, we will use Grid Search.  
We will try all combinations of parameters and find the best one, that gives us the best score.  
This may be a little exhaustive, especially when we want to check a lot of parameters and values.  
  
Random Search   
We can choose to get random combinations of parameters and check the score on them.  
This will not be as accurate as Grid Search, but it will take less time.  

In [33]:
# train with grid search and get best parameters
from sklearn.model_selection import GridSearchCV

feature_arr = ct.fit_transform(X_test_temp, t_train_temp)
feature_labels = ct.get_feature_names_out(all_cols.tolist())
X_encoded = pd.DataFrame(feature_arr, columns=feature_labels)

hyper_parameters = {'penalty': ('l2', 'l1', 'elasticnet'), 'alpha':[0.0001, 0.001, 0.01, 0.1]}

gs_model = GridSearchCV(SGDRegressor(random_state=1), hyper_parameters).fit(X_encoded, t_train_temp)
print('Accuracy score for regression:')
print('gs_model', gs_model.best_score_)
print('best params', gs_model.best_params_)

In [34]:
# train with random search and get best parameters
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

np.random.seed(1)
distributions = dict(alpha=uniform(loc=0, scale=1), penalty=['l2', 'l1', 'elasticnet'])

rs_model = RandomizedSearchCV(SGDRegressor(random_state=1), distributions, random_state=1).fit(X_encoded, t)
print('Accuracy score for regression:')
print('rs_model', rs_model.best_score_)
print('best params', rs_model.best_params_)

With hyper paramter search we see we get a lower score than previouts methods, so in my opinion it's better not be considered.

### Test  
Now let's our test for prediction.

In [35]:
# func for filling a null column value
def fill_random_column(df, column_name):
    df_not_null = df[~df[column_name].isnull()]
    df_null = df[df[column_name].isnull()]
    df[column_name] = df[column_name].apply(lambda x: np.random.choice(df_not_null[column_name]) if pd.isnull(x) else x)

Creating our new features in the test

In [36]:
test_df_feat = test_df.copy()
test_df_feat = test_df_feat.drop(['Id'], axis=1)
test_df_feat['TotalSF'] = test_df_feat['TotalBsmtSF'] + test_df_feat['1stFlrSF'] + test_df_feat['2ndFlrSF']
test_df_feat['HouseAge'] = test_df_feat['YrSold'] - test_df_feat['YearBuilt']
test_df_feat.reset_index(drop=True, inplace=True)
test_cp_df = test_df_feat[best_features]


count_empty_values_in_each_column(test_cp_df)

Fill empty values in the test set

In [37]:
fill_random_column(test_cp_df,'MSZoning')
fill_random_column(test_cp_df,'Utilities')
fill_random_column(test_cp_df,'BsmtFinSF1')
fill_random_column(test_cp_df,'BsmtFinSF2')
fill_random_column(test_cp_df,'BsmtHalfBath')
fill_random_column(test_cp_df,'Functional')
fill_random_column(test_cp_df,'Exterior2nd')
fill_random_column(test_cp_df,'BsmtUnfSF')
fill_random_column(test_cp_df,'GarageCars')


test_cp_df

In [38]:
count_empty_values_in_each_column(test_cp_df)

#### PREDICTION

In [39]:
final_predict = SGD_reg_original.predict(test_cp_df)
final_predict

#### SUBMISSION

In [40]:
submission_df['SalePrice'] = final_predict
submission_df.to_csv('submission.csv', index = False)
print("Submited")

#### Submissions and Ledaerboard  

The best score I got is 0.17. 
In this work i learned a lot about feature selection and encoidng.  
I struggled with encoding methods, it took me some time to figure out what is the best way to make it work as it should and fit the models I'm creating.